In [1]:
import os

In [11]:
%pwd

'c:\\Users\\Omar\\Desktop\\Omar_Files\\Python_Analysis\\EndToEndMachineLearningProjectAutoMobilePrice'

In [3]:
os.chdir("../")

In [12]:
%pwd

'c:\\Users\\Omar\\Desktop\\Omar_Files\\Python_Analysis\\EndToEndMachineLearningProjectAutoMobilePrice'

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    train_data_path: Path
    test_data_path: Path

In [14]:
from src.AutoMobilePriceRegression.constants import *
from src.AutoMobilePriceRegression.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        #schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        #self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        #schema=self.schema.COLUMNS

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            train_data_path= config.train_data_path,
            test_data_path=config.test_data_path
        )

        return data_ingestion_config

In [16]:
import os
import urllib.request as request
from src.AutoMobilePriceRegression import logger
from src.AutoMobilePriceRegression.utils.common import get_size
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.pipeline import Pipeline
import pandas as pd

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            df=pd.read_csv(self.config.source_URL)
            df.to_csv(self.config.local_data_file,index=False,header=True)

            train_set,test_set=train_test_split(df,test_size=0.2,random_state=42)

            if not os.path.exists(self.config.train_data_path):
                train_set.to_csv(self.config.train_data_path,index=False,header=True)            
        
            if not os.path.exists(self.config.test_data_path):
                test_set.to_csv(self.config.test_data_path,index=False,header=True) 

            return(
                self.config.train_data_path,
                self.config.test_data_path
            )     


        logger.info("Inmgestion of the data is completed")
       
        

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e

[2024-07-05 17:49:10,575: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-05 17:49:10,589: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-05 17:49:10,592: INFO: common: created directory at: artifacts]
[2024-07-05 17:49:10,594: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-05 17:49:10,595: INFO: 1448228844: Inmgestion of the data is completed]
